In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

base_path = "/content/drive/MyDrive/Colab Notebooks/platform_subscribe/"

train = pd.read_csv(base_path + "train.csv")
test = pd.read_csv(base_path + "test.csv")
submission = pd.read_csv(base_path + "sample_submission.csv")

In [3]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.5 MB/s eta 0:00:00


In [4]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.0 MB/s eta 0:00:00


In [5]:
# Optuna Libraries
import optuna
from optuna import Trial
from optuna.samplers import TPESampler

from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 평가 지표
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

evulation_metric = f1_score

### data_scaling + optuna+second_best = ['average_time_per_learning_session', 'recent_learning_achievement'] + data_scaling + catboost

In [6]:
second_best = ['average_time_per_learning_session', 'recent_learning_achievement']
third_best = ['average_time_per_learning_session',
   'recent_learning_achievement',
   'average_login_time',
   'subscription_duration',
   'recent_login_time',
   'monthly_active_learning_days',
   'total_completed_courses',
   'abandoned_learning_sessions',
   'payment_pattern',
   'customer_inquiry_history']

In [7]:
X=train[third_best]
y=train.target
X

,average_time_per_learning_session,recent_learning_achievement,average_login_time,subscription_duration,recent_login_time,monthly_active_learning_days,total_completed_courses,abandoned_learning_sessions,payment_pattern,customer_inquiry_history
0,8.427187,68.360455,14.946163,13,14,18,16,3,5,4
1,72.646087,97.567322,18.453224,16,18,16,13,2,6,1
2,21.774492,94.358763,16.195228,22,1,13,14,3,7,0
3,42.659066,70.153228,17.628656,1,19,19,18,0,0,1
4,30.744287,81.917908,21.390656,4,5,19,10,2,0,3
...,...,...,...,...,...,...,...,...,...,...
9995,84.053558,64.966803,14.727623,22,29,18,16,2,1,1
9996,45.464833,82.750244,19.374054,10,11,9,8,3,7,2
9997,127.302411,81.567839,18.240978,7,27,24,14,3,6,1
9998,5.297234,89.885656,18.783800,11,7,10,10,4,0,2


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.25, random_state=42)

In [9]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(X_train)
X_train_scaled = std.transform(X_train)
X_val_scaled = std.transform(X_val)

In [10]:
# random sampler
sampler = TPESampler(seed=10)

# define function
def objective(trial):

    cbrm_param = {
        'iterations':trial.suggest_int("iterations", 4000, 25000),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'subsample': trial.suggest_uniform('subsample',0,1),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'colsample_bylevel':trial.suggest_float('colsample_bylevel', 0.4, 1.0),
    }

    # Generate model
    model_cbclf = CatBoostClassifier(**cbrm_param)
    model_cbclf = model_cbclf.fit(X_train_scaled, y_train, eval_set=[(X_val, y_val)],
                           verbose=0, early_stopping_rounds=25)
    pred_val_cbclf=model_cbclf.predict(X_val_scaled)
    evulation_metric_val_cb = evulation_metric(y_val, pred_val_cbclf)
    return evulation_metric_val_cb

In [11]:
optuna_cbclf = optuna.create_study(direction="maximize", sampler=sampler)
optuna_cbclf.optimize(objective, n_trials=50)

[I 2023-12-10 09:32:57,934] A new study created in memory with name: no-name-290260e4-2688-4fb4-be65-987767958486
<ipython-input-10-c41b749777e4>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
<ipython-input-10-c41b749777e4>:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
<ipython-input-10-c41b749777e4>:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample',0,1),
<ipython-input-10-c41b749777e4>:

In [12]:
cbrm_trial = optuna_cbclf.best_trial
cbrm_trial_params = cbrm_trial.params

cbrm_trial_params

{'iterations': 17013,
 'od_wait': 2283,
 'learning_rate': 0.5770504637008435,
 'reg_lambda': 71.16145519119323,
 'subsample': 0.1799036241592752,
 'random_strength': 14.883669687068831,
 'depth': 3,
 'min_data_in_leaf': 30,
 'leaf_estimation_iterations': 15,
 'bagging_temperature': 0.03957711445834419,
 'colsample_bylevel': 0.8312782911257619}

In [13]:
best_model = CatBoostClassifier(**cbrm_trial_params, verbose=False)
best_model.fit(X_train_scaled, y_train)
pred_val_cbclf=best_model.predict(X_val_scaled)

evulation_metric_val_cb = evulation_metric(y_val, pred_val_cbclf)
evulation_metric_val_cb

0.6394984326018809

### test 확인

In [14]:
X_test_scaled = std.transform(X_test)
pred_test_cb=best_model.predict(X_test_scaled)

evulation_metric_test_cb=evulation_metric(y_test, pred_test_cb)
print(evulation_metric_test_cb)

0.629817049435578


### submission

In [15]:
X_submission=test[third_best]
X_submission_scaled = std.transform(X_submission)
print(X_submission_scaled)
preds=best_model.predict(X_submission_scaled)
preds

[[-0.87958133  0.71317603 -1.29045748 ... -0.01304575  1.49311635
  -0.01134425]
 [-0.56081826  0.55890233 -0.52631231 ...  0.56250208  0.2031051
   1.39206767]
 [-0.97664269 -0.13439635  1.18617732 ...  1.13804991 -0.22689864
  -0.7130502 ]
 ...
 [-0.6378283   1.44183736  0.6786404  ...  1.13804991 -1.51690989
  -0.7130502 ]
 [ 1.8374985   0.72769309  0.24926551 ... -0.58859358  0.63310885
  -0.01134425]
 [-0.0327581  -0.2759617   1.54560916 ... -0.58859358 -0.22689864
  -0.7130502 ]]


array([0, 1, 0, ..., 0, 1, 0])

In [16]:
submission['target']=preds
submission

,user_id,target
0,0001d6e9,0
1,0002c77d,1
2,0002df5b,0
3,000b6068,1
4,00184a0c,1
...,...,...
9995,ffe2eba5,1
9996,ffe710f1,1
9997,ffeccdef,0
9998,fff3fcea,1


In [17]:
unique, counts = np.unique(preds, return_counts = True)
cnt_dict = dict(zip(unique, counts))
cnt_dict

{0: 3481, 1: 6519}

In [18]:
submission.to_csv(base_path+"submission_1210_7.csv", index=False)